In [36]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import time
import json
from ultralytics import YOLO


In [2]:
frame_path = Path("frames")

# for frame in frame_path.iterdir():
#     img = cv2.imread(str(frame))
    
#     cv2.imshow("frame", img)
#     cv2.waitKey(0)

# cv2.destroyAllWindows()

In [4]:
img = cv2.imread(str(frame_path / "2024_05_07_13_41_0.png"))

height, width, _ = img.shape
print(height, width)

350 800


In [5]:

weight_path = Path("train_results/best.pt")
frames_path = Path("frames")
model = YOLO(weight_path)

## test iterated frames for output

In [9]:
#iterating through the frames


i = 0
for frame in frames_path.iterdir():
    i += 1
    img = cv2.imread(str(frame))
    results = model(img)
    result = results[0]
    # print the boxes
    # print(result.boxes)
    xywhn = result.boxes.xywhn
    l, r, c = 0, 0, 0
    for xywh in xywhn:
        x, y, w, h = xywh.cpu().numpy()
        x, y, w, h = x * width, y * height, w * width, h * height
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)

        # if lower bound is below 40% of the image height, it's a parked car
        if y2 > 0.4 * height:
            # check in which third of the image width the center of the is
            if x < 0.33 * width:
                l = 1
                print("Left")
            elif x < 0.66 * width:
                print("Middle")
                c = 1
            else:
                print("Right")
                r = 1

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow("frame", img)
    cv2.waitKey(0)
  
    if i == 1:
        break

cv2.destroyAllWindows()


0: 352x800 2 parked-cars, 142.8ms
Speed: 0.0ms preprocess, 142.8ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right


## Check random frames for output

In [16]:
# random frames
import random

frames = list(frames_path.iterdir())
random.shuffle(frames)
count = 10

for frame in frames:
    img = cv2.imread(str(frame))
    results = model(img)
    result = results[0]
    # print the boxes
    # print(result.boxes)
    xywhn = result.boxes.xywhn
    # display the image
    #plot xy points on the image
    l, r, c = 0, 0, 0
    for xywh in xywhn:
        x, y, w, h = xywh.cpu().numpy()
        x, y, w, h = x * width, y * height, w * width, h * height
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)

        # if lower bound is below 40% of the image height, it's a parked car
        if y2 > 0.4 * height:
            # check in which third of the image width the center of the cart is
            if x < 0.33 * width:
                l = 1
                print("Left")
            elif x < 0.66 * width:
                print("Middle")
                c = 1
            else:
                print("Right")
                r = 1

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow(f'left:{l}, center: {c}, right: {r}', img)
    
    cv2.waitKey(0)
    count -= 1
    if count == 0:
        break

cv2.destroyAllWindows()



0: 352x800 2 parked-cars, 169.2ms
Speed: 1.0ms preprocess, 169.2ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 119.6ms
Speed: 0.0ms preprocess, 119.6ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 3 parked-cars, 123.0ms
Speed: 1.0ms preprocess, 123.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Left
Right

0: 352x800 2 parked-cars, 115.6ms
Speed: 0.0ms preprocess, 115.6ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 101.3ms
Speed: 1.0ms preprocess, 101.3ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 100.2ms
Speed: 0.0ms preprocess, 100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Right
Middle

0: 352x800 2 parked-cars, 109.8ms
Speed: 0.0ms preprocess, 109.8ms inference, 0.0ms postprocess per image at shape (1,

## Proper function to jsonify output

In [39]:
def frame_to_json(frame, verbose=False):
    img = cv2.imread(str(frame))
    results = model(img, verbose=verbose)
    result = results[0]
    # print the boxes
    # print(result.boxes)
    xywhn = result.boxes.xywhn
    # display the image
    #plot xy points on the image
    l, r, c = 0, 0, 0
    for xywh in xywhn:
        x, y, w, h = xywh.cpu().numpy()
        x, y, w, h = x * width, y * height, w * width, h * height
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)

        # if lower bound is below 35% of the image height, it's a parked car
        if y2 > 0.35 * height:
            # check in which third of the image width the car is
            if x < 0.33 * width:
                l = 1
            elif x < 0.66 * width:
                c = 1
            else:
                r = 1

    if verbose:
        print(f"Left: {l}, Right: {r}, Center: {c}")
    name_split = frame.name.split("_")
    year, month, day, hour, minute, second = name_split[:6]
    second = str(int(second.split(".")[0]) * 10)
    return {
        "year": year,
        "month": month,
        "day": day,
        "hour": hour,
        "minute": minute,
        "second": second,
        "left": l,
        "right": r,
        "center": c
    }

In [40]:
count = 0
for frame in frames_path.iterdir():
    json_reult = frame_to_json(frame)
    print(json_reult)
    count += 1
    if count == 10:
        break

{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '0', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '10', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '20', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '30', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '40', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '50', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '42', 'second': '0', 'left': 0, 'right': 1, 'center': 1}
{'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '42', 'second': '10', 'left': 0, 'right': 1,

In [41]:
# iterate through all the frames and save the results in a json file in results folder
# speed about 9 frames per second
#1.5h to process the whole folder, only to do on final model
results = []
i = 0
for frame in frames_path.iterdir():
    json_reult = frame_to_json(frame)
    results.append(json_reult)
    i += 1

    ############################################
    # break after 100 frames
    if i == 100:
        break
    ############################################
    
    if i % 1000 == 0:
        print(f"Processed {i} frames")

with open("results/results.json", "w") as f:
    json.dump(results, f)

    

In [38]:
# prettifing the json file
with open("results/results.json", "r") as f:
    results = json.load(f)

with open("results/results_pretty.json", "w") as f:
    json.dump(results, f, indent=4)

    